# Day17_1: RNN, LSTM, GRU (시퀀스 모델링) - 정답

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

## Q1. RNN 출력 shape 계산하기

**문제**: 다음 RNN의 출력 shape을 계산하세요.

```python
rnn = nn.RNN(input_size=50, hidden_size=100, num_layers=2, batch_first=True)
x = torch.randn(16, 20, 50)  # batch=16, seq=20, input=50
outputs, h_n = rnn(x)
```

In [ ]:
# 정답 코드
rnn = nn.RNN(input_size=50, hidden_size=100, num_layers=2, batch_first=True)
x = torch.randn(16, 20, 50)  # batch=16, seq=20, input=50

outputs, h_n = rnn(x)

print(f"outputs shape: {outputs.shape}")
print(f"h_n shape: {h_n.shape}")

In [ ]:
# 테스트/검증
assert outputs.shape == torch.Size([16, 20, 100]), "outputs shape이 잘못되었습니다"
assert h_n.shape == torch.Size([2, 16, 100]), "h_n shape이 잘못되었습니다"
print("검증 통과!")

### 풀이 설명

**접근 방법**:
- RNN의 출력 shape을 이해하려면 각 차원의 의미를 알아야 합니다.

**핵심 개념**:
- `outputs`: 모든 시점의 마지막 레이어 hidden state
  - shape: (batch, seq_len, hidden_size) = (16, 20, 100)
- `h_n`: 모든 레이어의 마지막 시점 hidden state
  - shape: (num_layers, batch, hidden_size) = (2, 16, 100)

**실무 팁**:
- `batch_first=True` 설정 시 입력/출력의 첫 번째 차원이 batch가 됩니다.
- 분류 문제에서는 보통 `h_n[-1]` (마지막 레이어의 마지막 hidden)을 사용합니다.

---

## Q2. Hidden State 이해하기

**문제**: RNN의 마지막 시점 출력(outputs[:, -1, :])과 h_n이 같은지 확인하세요.

In [ ]:
# 정답 코드
# 단일 레이어 RNN
rnn_single = nn.RNN(input_size=10, hidden_size=20, num_layers=1, batch_first=True)
x = torch.randn(4, 5, 10)  # batch=4, seq=5, input=10

outputs, h_n = rnn_single(x)

print(f"outputs[:, -1, :] shape: {outputs[:, -1, :].shape}")
print(f"h_n.squeeze(0) shape: {h_n.squeeze(0).shape}")

# 같은지 확인
is_same = torch.allclose(outputs[:, -1, :], h_n.squeeze(0))
print(f"\noutputs[:, -1, :] == h_n.squeeze(0): {is_same}")

In [ ]:
# 테스트/검증
# 다층 RNN에서는 어떨까?
rnn_multi = nn.RNN(input_size=10, hidden_size=20, num_layers=2, batch_first=True)
outputs_m, h_n_m = rnn_multi(x)

# outputs[:, -1, :]는 마지막 레이어의 마지막 시점
# h_n[-1]도 마지막 레이어의 마지막 시점
is_same_multi = torch.allclose(outputs_m[:, -1, :], h_n_m[-1])
print(f"다층 RNN에서도 같은가?: {is_same_multi}")

### 풀이 설명

**접근 방법**:
- `outputs`와 `h_n`의 관계를 이해합니다.

**핵심 개념**:
- `outputs[:, -1, :]`: 마지막 레이어의 마지막 시점 hidden state
- `h_n[-1]`: 마지막 레이어의 마지막 시점 hidden state
- 둘은 동일합니다!

**주의사항**:
- 다층 RNN에서 `h_n`은 모든 레이어의 마지막 시점을 포함합니다.
- 분류에서는 `h_n[-1]`을 사용하는 것이 일반적입니다.

---

## Q3. 시퀀스 데이터 특성

**문제**: 다음 중 시퀀스 데이터가 아닌 것을 고르고 이유를 설명하세요.

In [ ]:
# 정답
answer = """
정답: 3. 학생들의 키와 몸무게

이유:
- 시퀀스 데이터는 순서가 중요한 데이터입니다.
- 학생들의 키와 몸무게는 개별 학생의 특성으로, 순서에 의미가 없습니다.
- 학생 A, B, C의 순서를 바꿔도 데이터의 의미는 변하지 않습니다.

다른 항목들이 시퀀스인 이유:
1. 일별 주가: 시간 순서가 중요 (오늘 주가는 어제에 영향받음)
2. 영화 리뷰 텍스트: 단어 순서가 의미를 결정 ("not good" vs "good not")
4. 음성 파형: 시간에 따른 소리의 변화, 순서가 필수
"""
print(answer)

### 풀이 설명

**핵심 개념**:
- 시퀀스 데이터: 순서(order)가 의미를 갖는 데이터
- 정형 데이터(tabular): 각 샘플이 독립적, 순서 무관

**실무 팁**:
- 시퀀스 데이터 -> RNN, LSTM, GRU, Transformer
- 정형 데이터 -> MLP, Decision Tree, XGBoost 등

---

## Q4. LSTM 게이트 이해하기

**문제**: LSTM의 3가지 게이트(Forget, Input, Output)의 역할을 각각 한 문장으로 설명하세요.

In [ ]:
# 정답
answer = """
LSTM의 3가지 게이트:

1. Forget Gate (망각 게이트):
   - 역할: 이전 Cell State에서 어떤 정보를 버릴지 결정합니다.
   - 수식: f_t = sigmoid(W_f * [h_{t-1}, x_t] + b_f)
   - 출력 범위: 0~1 (0이면 완전히 잊음, 1이면 완전히 기억)

2. Input Gate (입력 게이트):
   - 역할: 새로운 정보 중 어떤 것을 Cell State에 저장할지 결정합니다.
   - 수식: i_t = sigmoid(W_i * [h_{t-1}, x_t] + b_i)
   - 새 후보 값: C_tilde = tanh(W_C * [h_{t-1}, x_t] + b_C)

3. Output Gate (출력 게이트):
   - 역할: Cell State 중 어떤 부분을 Hidden State로 출력할지 결정합니다.
   - 수식: o_t = sigmoid(W_o * [h_{t-1}, x_t] + b_o)
   - Hidden State: h_t = o_t * tanh(C_t)

Cell State 업데이트:
   C_t = f_t * C_{t-1} + i_t * C_tilde
   (잊을 것 버리고 + 새로 저장할 것 추가)
"""
print(answer)

### 풀이 설명

**직관적 이해**:
- **Forget Gate**: "이전에 배운 것 중 뭘 잊을까?"
- **Input Gate**: "새로 배운 것 중 뭘 기억할까?"
- **Output Gate**: "기억한 것 중 뭘 말할까?"

**실무 팁**:
- LSTM은 Cell State라는 '장기 기억 통로'를 통해 정보가 손실 없이 흐를 수 있습니다.
- 게이트는 모두 sigmoid(0~1)로 '얼마나'를 결정합니다.

---

## Q5. GRU와 LSTM 차이

**문제**: GRU가 LSTM보다 파라미터 수가 적은 이유를 설명하세요.

In [ ]:
# 정답
answer = """
GRU가 LSTM보다 파라미터가 적은 이유:

1. 게이트 수 차이:
   - LSTM: 3개 (Forget, Input, Output) + Cell State 계산
   - GRU: 2개 (Reset, Update)

2. 상태 수 차이:
   - LSTM: Hidden State + Cell State (2개)
   - GRU: Hidden State만 (1개)

3. 파라미터 계산:
   - LSTM: 4개의 가중치 행렬 (forget, input, cell, output)
     -> 4 * (input_size + hidden_size) * hidden_size
   - GRU: 3개의 가중치 행렬 (reset, update, hidden)
     -> 3 * (input_size + hidden_size) * hidden_size

결론: GRU는 LSTM의 약 75% 파라미터로 비슷한 성능을 낼 수 있어,
     학습 속도가 빠르고 메모리 효율적입니다.
"""
print(answer)

In [ ]:
# 실제 파라미터 수 비교
input_size, hidden_size = 100, 128

lstm = nn.LSTM(input_size, hidden_size)
gru = nn.GRU(input_size, hidden_size)

lstm_params = sum(p.numel() for p in lstm.parameters())
gru_params = sum(p.numel() for p in gru.parameters())

print(f"LSTM 파라미터: {lstm_params:,}")
print(f"GRU 파라미터: {gru_params:,}")
print(f"비율: GRU/LSTM = {gru_params/lstm_params:.2%}")

---

## Q6. 양방향 RNN

**문제**: 양방향 LSTM을 정의하고, 출력 shape을 확인하세요.

In [ ]:
# 정답 코드
bi_lstm = nn.LSTM(
    input_size=30,
    hidden_size=64,
    num_layers=1,
    batch_first=True,
    bidirectional=True
)

# 입력
x = torch.randn(8, 15, 30)  # batch=8, seq=15, input=30

# 순전파
outputs, (h_n, c_n) = bi_lstm(x)

print(f"outputs shape: {outputs.shape}")
print(f"h_n shape: {h_n.shape}")
print(f"c_n shape: {c_n.shape}")

In [ ]:
# 테스트/검증
assert outputs.shape == torch.Size([8, 15, 128]), "outputs shape 오류 (hidden*2=128)"
assert h_n.shape == torch.Size([2, 8, 64]), "h_n shape 오류 (num_layers*2=2)"
print("검증 통과!")

### 풀이 설명

**핵심 개념**:
- `bidirectional=True` 설정 시:
  - `outputs` 차원: hidden_size * 2 (순방향 + 역방향 concat)
  - `h_n` 첫 번째 차원: num_layers * 2

**출력 shape 해석**:
- outputs: (8, 15, 128) = (batch, seq, hidden*2)
- h_n: (2, 8, 64) = (방향 수, batch, hidden)
  - h_n[0]: 순방향 마지막 hidden
  - h_n[1]: 역방향 마지막 hidden

**실무 팁**:
- 분류 시 순방향+역방향 concat: `torch.cat([h_n[0], h_n[1]], dim=1)`

---

## Q7. 시계열 데이터 전처리

**문제**: 주가 데이터로 시퀀스 길이 5인 학습 데이터를 생성하세요.

In [ ]:
# 정답 코드
prices = [100, 102, 105, 103, 108, 110, 107, 112, 115, 113]
seq_length = 5

# 시퀀스 데이터 생성
X = []
y = []

for i in range(len(prices) - seq_length):
    X.append(prices[i:i+seq_length])  # 과거 5일
    y.append(prices[i+seq_length])    # 다음 날

X = np.array(X)
y = np.array(y)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n시퀀스 예시:")
for i in range(len(X)):
    print(f"  입력: {X[i]} -> 예측: {y[i]}")

In [ ]:
# 테스트/검증
assert X.shape == (5, 5), "X shape이 (5, 5)여야 합니다"
assert y.shape == (5,), "y shape이 (5,)여야 합니다"
assert list(X[0]) == [100, 102, 105, 103, 108], "첫 번째 시퀀스 확인"
assert y[0] == 110, "첫 번째 타겟 확인"
print("검증 통과!")

### 풀이 설명

**접근 방법**:
- 슬라이딩 윈도우 방식으로 시퀀스 생성
- 과거 N일 -> 다음 1일 예측

**핵심 개념**:
```
원본: [100, 102, 105, 103, 108, 110, 107, 112, 115, 113]
         |----시퀀스 1----|  타겟1
              |----시퀀스 2----|  타겟2
                   |----시퀀스 3----|  타겟3
                        ....
```

**실무 팁**:
- 시계열 예측에서는 미래 데이터 누출(data leakage)에 주의!
- 훈련/테스트 분할 시 시간 순서 유지 필수

---

## Q8. 임베딩 레이어 이해

**문제**: Embedding 레이어의 출력 shape을 확인하세요.

In [ ]:
# 정답 코드
vocab_size = 1000
embed_dim = 64

# 임베딩 레이어 생성
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)

# 입력: 토큰 인덱스 (batch=4, seq=10)
# 각 값은 0 ~ vocab_size-1 사이의 정수
x = torch.randint(0, vocab_size, (4, 10))

# 임베딩 적용
embedded = embedding(x)

print(f"입력 shape: {x.shape}")
print(f"입력 dtype: {x.dtype}")
print(f"\n출력 shape: {embedded.shape}")
print(f"출력 dtype: {embedded.dtype}")

In [ ]:
# 테스트/검증
assert embedded.shape == torch.Size([4, 10, 64]), "출력 shape 오류"
print("검증 통과!")

# 임베딩 가중치 확인
print(f"\n임베딩 가중치 shape: {embedding.weight.shape}")
print(f"파라미터 수: {embedding.weight.numel():,}")

### 풀이 설명

**핵심 개념**:
- Embedding: 정수 인덱스 -> 실수 벡터 변환 (룩업 테이블)
- 입력: (batch, seq_len) - LongTensor
- 출력: (batch, seq_len, embed_dim) - FloatTensor

**작동 원리**:
```
vocab_size=1000, embed_dim=64
-> 1000x64 행렬 (각 단어별 64차원 벡터)

입력 [3, 7, 2] -> 3번 행, 7번 행, 2번 행 추출
```

**실무 팁**:
- 사전 학습된 임베딩(Word2Vec, GloVe, FastText) 로드 가능
- `padding_idx` 설정으로 패딩 토큰 처리

---

## Q9. 주가 예측 LSTM 구현

**문제**: 사인파 데이터로 다음 값을 예측하는 LSTM 모델을 구현하세요.

In [ ]:
# 정답 코드
from sklearn.preprocessing import MinMaxScaler

# 사인파 데이터
np.random.seed(42)
x_data = np.linspace(0, 20*np.pi, 500)
y_data = np.sin(x_data) + 0.1 * np.random.randn(500)

# 정규화
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(y_data.reshape(-1, 1))

# 시퀀스 생성
SEQ_LENGTH = 20

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, SEQ_LENGTH)

# 훈련/테스트 분할
train_size = int(len(X) * 0.8)
X_train = torch.FloatTensor(X[:train_size])
y_train = torch.FloatTensor(y[:train_size])
X_test = torch.FloatTensor(X[train_size:])
y_test = torch.FloatTensor(y[train_size:])

print(f"훈련 데이터: {X_train.shape}, {y_train.shape}")
print(f"테스트 데이터: {X_test.shape}, {y_test.shape}")

In [ ]:
# LSTM 모델 정의
class SineLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# 모델 생성
model = SineLSTM(input_size=1, hidden_size=32, num_layers=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(model)

In [ ]:
# 학습
epochs = 30
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

losses = []
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

In [ ]:
# 예측 및 시각화
model.eval()
with torch.no_grad():
    train_pred = model(X_train).numpy()
    test_pred = model(X_test).numpy()

# 역정규화
train_pred_inv = scaler.inverse_transform(train_pred)
test_pred_inv = scaler.inverse_transform(test_pred)
y_train_inv = scaler.inverse_transform(y_train.numpy())
y_test_inv = scaler.inverse_transform(y_test.numpy())

# 시각화
fig = go.Figure()

# 전체 실제 데이터
fig.add_trace(go.Scatter(y=y_data, mode='lines', name='실제값', line=dict(color='blue')))

# 테스트 예측
test_idx = list(range(train_size + SEQ_LENGTH, len(y_data)))
fig.add_trace(go.Scatter(x=test_idx, y=test_pred_inv.flatten(), 
                          mode='lines', name='LSTM 예측', line=dict(color='red', dash='dash')))

fig.add_vline(x=train_size + SEQ_LENGTH, line_dash='dot', line_color='green',
              annotation_text='Train/Test')

fig.update_layout(title='사인파 LSTM 예측 결과', xaxis_title='시점', yaxis_title='값',
                  template='plotly_white')
fig.show()

### 풀이 설명

**접근 방법**:
1. 데이터 정규화 (MinMaxScaler)
2. 시퀀스 데이터 생성 (슬라이딩 윈도우)
3. LSTM 모델 정의 및 학습
4. 예측 후 역정규화

**핵심 개념**:
- 시계열 예측에서 정규화는 필수
- LSTM의 마지막 시점 출력 -> FC -> 예측값

**실무 팁**:
- 복잡한 시계열은 다층 LSTM 또는 양방향 사용
- Dropout 추가로 과적합 방지

---

## Q10. 감성 분석 모델 개선

**문제**: SentimentLSTM 모델을 GRU로 변경하고 개선하세요.

In [ ]:
# 데이터 준비 (본문과 동일)
vocab = {
    '<PAD>': 0, '<UNK>': 1,
    'this': 2, 'movie': 3, 'was': 4, 'is': 5,
    'great': 6, 'good': 7, 'bad': 8, 'terrible': 9,
    'amazing': 10, 'awful': 11, 'boring': 12, 'exciting': 13,
    'the': 14, 'a': 15, 'very': 16, 'really': 17,
    'loved': 18, 'hated': 19, 'it': 20, 'film': 21
}

positive_reviews = ["this movie was great", "the film is amazing", "really good movie",
                    "loved this film", "very exciting movie"]
negative_reviews = ["this movie was bad", "the film is terrible", "really awful movie",
                    "hated this film", "very boring movie"]

def text_to_indices(text, vocab, max_len=10):
    tokens = text.lower().split()
    indices = [vocab.get(t, vocab['<UNK>']) for t in tokens]
    if len(indices) < max_len:
        indices = indices + [vocab['<PAD>']] * (max_len - len(indices))
    return indices[:max_len]

X_data = [text_to_indices(r, vocab) for r in positive_reviews + negative_reviews]
y_data = [1]*5 + [0]*5
X_data = X_data * 20
y_data = y_data * 20

X_tensor = torch.LongTensor(X_data)
y_tensor = torch.FloatTensor(y_data).reshape(-1, 1)

In [ ]:
# 정답 코드: 개선된 GRU 모델
class ImprovedSentimentGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=2, dropout=0.3):
        super().__init__()
        # 임베딩
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # GRU (2층, 양방향, Dropout)
        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # 분류 레이어 (양방향이므로 hidden_size * 2)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # 임베딩
        embedded = self.embedding(x)  # (batch, seq, embed)
        embedded = self.dropout(embedded)
        
        # GRU
        gru_out, h_n = self.gru(embedded)
        # h_n: (num_layers*2, batch, hidden)
        
        # 마지막 레이어의 순방향+역방향 결합
        forward_h = h_n[-2]  # 순방향 마지막 레이어
        backward_h = h_n[-1]  # 역방향 마지막 레이어
        hidden = torch.cat([forward_h, backward_h], dim=1)
        hidden = self.dropout(hidden)
        
        # 분류
        out = self.fc(hidden)
        return self.sigmoid(out)

# 모델 생성
improved_model = ImprovedSentimentGRU(
    vocab_size=len(vocab),
    embed_dim=32,
    hidden_size=64,
    num_layers=2,
    dropout=0.3
)

print(improved_model)

In [ ]:
# 학습
criterion = nn.BCELoss()
optimizer = optim.Adam(improved_model.parameters(), lr=0.01)

dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

epochs = 50
for epoch in range(epochs):
    improved_model.train()
    total_loss = 0
    
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        output = improved_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

In [ ]:
# 테스트
def predict_improved(text, model, vocab):
    model.eval()
    indices = text_to_indices(text, vocab)
    x = torch.LongTensor([indices])
    with torch.no_grad():
        prob = model(x).item()
    return "긍정" if prob > 0.5 else "부정", prob

test_reviews = [
    "this movie was great",
    "the film is terrible",
    "really loved it",
    "very bad film",
    "amazing movie",
    "awful boring film"
]

print("개선된 GRU 모델 감성 분석 결과:")
print("="*50)
for review in test_reviews:
    sentiment, prob = predict_improved(review, improved_model, vocab)
    print(f"'{review}'")
    print(f"  -> {sentiment} (확률: {prob:.3f})")

### 풀이 설명

**개선 사항**:
1. **LSTM -> GRU**: 파라미터 감소, 학습 속도 향상
2. **Dropout 추가**: 과적합 방지
3. **2층 구조**: 더 복잡한 패턴 학습
4. **양방향 유지**: 텍스트 분류에 효과적

**핵심 개념**:
- GRU는 LSTM보다 25% 적은 파라미터로 비슷한 성능
- 다층 RNN에서는 레이어 사이에 dropout 적용 가능
- 양방향 출력은 마지막 레이어의 순방향+역방향 결합

**실무 팁**:
- 실제로는 사전 학습된 임베딩(Word2Vec, FastText) 사용
- 긴 텍스트는 BERT 같은 Transformer 모델이 더 효과적

---

## 학습 정리

### 핵심 개념 요약

| 퀴즈 | 핵심 개념 | 실무 포인트 |
|------|----------|------------|
| Q1 | RNN 출력 shape | batch_first, num_layers 이해 |
| Q2 | Hidden State | outputs[:,-1,:] == h_n[-1] |
| Q3 | 시퀀스 데이터 | 순서의 의미 |
| Q4 | LSTM 게이트 | Forget, Input, Output |
| Q5 | GRU vs LSTM | 파라미터 75%, 비슷한 성능 |
| Q6 | 양방향 RNN | 출력 차원 2배 |
| Q7 | 시퀀스 생성 | 슬라이딩 윈도우 |
| Q8 | Embedding | 정수 -> 벡터 변환 |
| Q9 | 시계열 예측 | 정규화, LSTM, 시각화 |
| Q10 | 모델 개선 | GRU, Dropout, 다층 |

### 실전 팁

1. **모델 선택**: GRU로 시작, 성능 부족 시 LSTM
2. **전처리**: 시계열은 정규화 필수, 텍스트는 토큰화+임베딩
3. **과적합 방지**: Dropout, Early Stopping
4. **시각화**: 예측 vs 실제 비교로 모델 성능 직관적 파악